In [1]:
import os
import requests
from dotenv import load_dotenv
# from langchain.tools import StructuredTool, BaseTool, tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import ToolException
from pydantic import ValidationError
from typing import Any, Type, List, Dict
import datetime

from openai import AzureOpenAI
from agency_swarm import set_openai_client
from agency_swarm import Agent, Agency


from composio.tools.local.clickup.actions.base import OpenAPIAction
from composio.tools.local.clickup.actions.create_task import CreateTask, CreateTaskResponse

from agents.ClickUpAgent.tools.ClickUpTools import initialize_clickup_tools


load_dotenv()

True

In [2]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    timeout=5,
    max_retries=5,
)

set_openai_client(client)

In [3]:
project_manager = Agent(
    name="project_manager",
    description="I am the Project Manager",
    model="gpt-4o",
    )

In [4]:
clickup_agent_instructions = f"""# PROJECT_MANAGEMENT
Your team uses ClickUp to manage projects and tasks.
Users often refer to lists and tasks by name, and sometimes there may not be an exact match, so you should look for the closest ones.
If in doubt, provide the user with valid options to choose from or ask for more information if necessary.

## CLICKUP_STRUCTURE
-> Workspace: The highest level of organization in ClickUp. Contains all your Spaces.
--> Space: A collection of Folders and Lists. It is a way to group related work.
---> Folder: Used to group Lists.
----> List: Used to group tasks. Lists can be in a Folder or directly in a Space.
-----> Task: The basic unit of work in ClickUp, assignable with due dates.
------> Subtask: A child task of a parent Task, assignable to different people.

## DEFAULT_CLICKUP_IDS
Use these IDs unless specified otherwise
- Team ID: {os.getenv("CLICKUP_TEAM_ID")}
- List ID 'test_clickup_tool': {os.getenv("CLICKUP_LIST_ID")}

## CLICKUP_USERS
- Your id: {os.getenv("CLICKUP_AGENT_USER_ID")}
- Main user id: {os.getenv("CLICKUP_USER_ID")}
"""

In [5]:
# clickup_tools = initialize_clickup_tools()
# print(clickup_tools)

[<class 'utils.tool_wrapper._convert_function_tool.<locals>.ConvertedFunctionTool'>, <class 'utils.tool_wrapper._convert_function_tool.<locals>.ConvertedFunctionTool'>, <class 'utils.tool_wrapper._convert_function_tool.<locals>.ConvertedFunctionTool'>, <class 'utils.tool_wrapper._convert_function_tool.<locals>.ConvertedFunctionTool'>, <class 'utils.tool_wrapper._convert_function_tool.<locals>.ConvertedFunctionTool'>, <class 'utils.tool_wrapper._convert_function_tool.<locals>.ConvertedFunctionTool'>, <class 'utils.tool_wrapper._convert_function_tool.<locals>.ConvertedFunctionTool'>, <class 'utils.tool_wrapper._convert_function_tool.<locals>.ConvertedFunctionTool'>]


In [ ]:
from agents.ClickUpAgent.tools.GetTaskTool import GetTaskTool
from utils.tool_wrapper import convert_langchain_tools
get_task_tool = GetTaskTool()
clickup_tools = convert_langchain_tools([get_task_tool])

In [6]:
clickup_tools[0].__name__

'Create_task_toolTool'

In [7]:

clickup_agent = Agent(
    name="clickup_agent",
    description="I am a simple agent",
    instructions=clickup_agent_instructions,
    model="gpt-4o",
    tools=clickup_tools,
    )

In [8]:
agency = Agency(
    agency_chart=[
    project_manager,
    [project_manager, clickup_agent],
],
    # shared_instructions="agency_manifesto.md"
)

Updating agent... clickup_agent


In [9]:
agency.demo_gradio()

/Users/morossini/Projects/professional_demos/agency_swarm_demo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/morossini/Projects/professional_demos/agency_swarm_demo/.venv/lib/python3.11/site-packages/gradio/components/chatbot.py:285: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Gradio Blocks instance: 6 backend functions
-------------------------------------------
fn_index=0
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x11f7e8c50>
 |-<gradio.components.chatbot.Chatbot object at 0x11f2cd950>
 outputs:
 |-<gradio.components.textbox.Textbox object at 0x11f7e8c50>
 |-<gradio.components.chatbot.Chatbot object at 0x11f2cd950>
fn_index=1
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x11f7e8c50>
 |-<gradio.components.chatbot.Chatbot object at 0x11f2cd950>
 |-<gradio.components.dropdown.Dropdown object at 0x123608550>
 outputs:
 |-<gradio.components.textbox.Textbox object at 0x11f7e8c50>
 |-<gradio.components.chatbot.Chatbot object at 0x11f2cd950>
 |-<gradio.components.dropdown.Dropdown object at 0x123608550>
fn_index=2
 inputs:
 |-<gradio.components.dropdown.Dropdown object at 0x123608550>
 outputs:
fn_index=3
 inputs:
 |-<gradio.templates.Files object at 0x122a95b50>
 outputs:
fn_index=4
 inputs:
 |-<gradio.components.textbox.Textbox object 

Message files:  []
Images:  []
THREAD:[ user -> project_manager ]: URL https://platform.openai.com/playground/assistants?assistant=asst_q3bAPsDDQmwfijHqK7v8Z655&mode=assistant&thread=thread_awYwQAYJsii7m4eCHOOtpQtX
THREAD:[ project_manager -> clickup_agent ]: URL https://platform.openai.com/playground/assistants?assistant=asst_NQWCwFwP37B8T5rATBm6Fi6E&mode=assistant&thread=thread_tT4NsmS9eZuM0FmYLzwfwHMl


[2025-02-27 18:56:41,742][ERROR] Tool execution failed: list_id is required for getting list information
Traceback (most recent call last):
  File "/Users/morossini/Projects/professional_demos/agency_swarm_demo/agency_ai_demo/utils/tool_wrapper.py", line 409, in run
    result = tool_func._run(**kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/morossini/Projects/professional_demos/agency_swarm_demo/agency_ai_demo/agents/ClickUpAgent/tools/ClickUpTools.py", line 1055, in _run
    raise ToolException("list_id is required for getting list information")
langchain_core.tools.base.ToolException: list_id is required for getting list information


...


==== GetTaskTool._run received parameters: ====
kwargs: {'kwargs': FieldInfo(annotation=NoneType, required=False, default=None, description='Parameter: kwargs')}


[2025-02-27 18:58:04,314][ERROR] Tool execution failed: task_id is required for getting a task
Traceback (most recent call last):
  File "/Users/morossini/Projects/professional_demos/agency_swarm_demo/agency_ai_demo/utils/tool_wrapper.py", line 409, in run
    result = tool_func._run(**kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/morossini/Projects/professional_demos/agency_swarm_demo/agency_ai_demo/agents/ClickUpAgent/tools/ClickUpTools.py", line 1299, in _run
    raise ToolException("task_id is required for getting a task")
langchain_core.tools.base.ToolException: task_id is required for getting a task



==== GetTaskTool._run received parameters: ====
kwargs: {'kwargs': FieldInfo(annotation=NoneType, required=False, default=None, description='Parameter: kwargs')}


[2025-02-27 18:58:08,100][ERROR] Tool execution failed: task_id is required for getting a task
Traceback (most recent call last):
  File "/Users/morossini/Projects/professional_demos/agency_swarm_demo/agency_ai_demo/utils/tool_wrapper.py", line 409, in run
    result = tool_func._run(**kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/morossini/Projects/professional_demos/agency_swarm_demo/agency_ai_demo/agents/ClickUpAgent/tools/ClickUpTools.py", line 1299, in _run
    raise ToolException("task_id is required for getting a task")
langchain_core.tools.base.ToolException: task_id is required for getting a task



==== GetTaskTool._run received parameters: ====
kwargs: {'kwargs': FieldInfo(annotation=NoneType, required=False, default=None, description='Parameter: kwargs')}
Message files:  []
Images:  []
THREAD:[ user -> project_manager ]: URL https://platform.openai.com/playground/assistants?assistant=asst_q3bAPsDDQmwfijHqK7v8Z655&mode=assistant&thread=thread_awYwQAYJsii7m4eCHOOtpQtX
THREAD:[ project_manager -> clickup_agent ]: URL https://platform.openai.com/playground/assistants?assistant=asst_NQWCwFwP37B8T5rATBm6Fi6E&mode=assistant&thread=thread_tT4NsmS9eZuM0FmYLzwfwHMl

==== CreateTaskTool._run received parameters: ====
kwargs: {'kwargs': FieldInfo(annotation=NoneType, required=False, default=None, description='Parameter: kwargs')}
No list_id found in parameters, using hardcoded value 901307715461
list_id being used: 901307715461
==== End parameters ====

URL being used: https://api.clickup.com/api/v2/list/901307715461/task
Request parameters: {'name': 'API TEST TASK', 'assignees': [8191895